

OData is actually a pretty inefficient interface when you want to sift through the entire collection of data
in ways not initially designed for, but there you go.


In [ ]:
!pip3 install tkapi

In [1]:
import tkapi, tkapi.document   # pip install tkapi ;   library with MIT license
from tkapi.document import DocumentSoort
api = tkapi.TKApi()

In [4]:
dossiers = api.get_dossiers( )
len(dossiers)

6851

In [18]:
for dossier in sorted(dossiers, key=lambda dos:str(dos.nummer)+(dos.toevoeging or ''))[:200]:
    # dossier numbers are apparently not unique without the toevoeging
    nummer_toevoeging = ('%s-%s'%(dossier.nummer, dossier.toevoeging or '')).rstrip('-')
    print( "== Dossier %s == %s =="%( nummer_toevoeging, dossier.titel) )
    print( '  ',dossier.url.replace(')','%29') ) # the replace is to make the notebook's url include the final bracket
    #for doci, document in enumerate(dossier.documenten):
    #    print( '   ',doci, 'bestand_url', document.bestand_url )

== Dossier 17050 == Misbruik en oneigenlijk gebruik op het gebied van belastingen, sociale zekerheid en subsidies ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(58cf1611-70a0-436a-ba73-f5e0819fd2a9%29
== Dossier 17257 == Assemblee van de West-Europese Unie ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(2af0d9a4-cccc-416b-870d-393e1299d540%29
== Dossier 18031 == Onderzoek militaire missie Suriname ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(a77d0458-31d5-43ca-8b6d-3061ec8f8efe%29
== Dossier 18106 == Voortgang rivierdijkversterkingen ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(3698cafd-4ffd-428c-9459-4aa205d98ef0%29
== Dossier 19291 == Parlementaire Assemblée van de NAVO ==
   https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Kamerstukdossier(b85f8944-7c73-47dc-8055-ed28bc901fc0%29
== Dossier 19637 == Vreemdelingenbeleid ==
   https://gegevensmagazijn.tweedekamer.n

In [37]:
sorted_dossiers = sorted(dossiers, key=lambda d:d.nummer, reverse=True )

for dossier in sorted_dossiers:
    nummer_toevoeging = ('%s-%s'%(dossier.nummer, dossier.toevoeging or '')).rstrip('-')

    if (dossier.nummer%100) == 0: # ignore some specific special cases for now, because they're large
        continue

    ## Select only dossiers that have Raad Van State advice
    #  aside from this being our interest 
    #  - this is also a good filter for just wetsvoorstellen
    #  - and filters out wetsvoorstellen that don't need this advice (e.g. begroting)
    #     
    # However, keep in mind
    # - there are other things that RVS advises on, like finances (e.g. 36200) 
    # - there are law changes that RVS does not advise on (e.g. )
    sorted_docs = sorted(dossier.documenten, key=lambda d:d.volgnummer )
    has_raadvanstate = False
    for document in sorted_docs:
        try:
            # these come from an enum, try  list( tkapi.document.DocumentSoort )  to see a list
            if document.soort in (DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE, 
                                  #DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_NADER_RAPPORT, # seems to be begrotingstuff?  (TODO: check)
                                  DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_REACTIE_VAN_DE_INITIATIEFNEMERS,
                                ):
                has_raadvanstate = True
        except ValueError: # there's some invalid / non-covered soort values in the data
            pass # ignore
        # we can filter on more, but we may not need to?
    if not has_raadvanstate:
        continue
    
    #Okay, it's interesting to us.
    #if len(sorted_docs)>500:
    #    print( "\n\n== Dossier %s == %s =="%( dossier.nummer, dossier.titel) )
    #    print(' LARGE: %d documents'%len(sorted_docs))
    #    print(' %s ({{kamerdossier|%d}}'%(dossier.titel, dossier.nummer))
    #    continue

    print( "\n== %r == Dossier %s == %d docs == %s =="%( dossier.id, nummer_toevoeging, len(dossier.documenten), dossier.titel) )
    for document in sorted_docs:
        try:
            if 0: # just to make the summaries a little easier to read
                if document.soort in (DocumentSoort.MOTIE, DocumentSoort.AMENDEMENT, DocumentSoort.BRIEF_REGERING, DocumentSoort.VERSLAG_VAN_EEN_ALGEMEEN_OVERLEG,
                                    DocumentSoort.MEMORIE_VAN_TOELICHTING_INITIATIEFVOORSTEL,
                                    ):
                    continue
        except ValueError:
            print( "soort not known by tkapi")
            continue

        try:
            dsoort = document.soort
        except ValueError: # this seems to be internal inconsistency
            continue

        if True or dsoort in (DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE, 
                      DocumentSoort.ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_REACTIE_VAN_DE_INITIATIEFNEMERS):

            print( '#%s'%(document.volgnummer, ), document.soort.name)
            #print( 'soort', document.soort.name, '(%s)'%document.soort.value )
            print( '  onderwerp', document.onderwerp )     # for wetsvoorstel-dossiers, seems to often be the same as soort plus some detail (who a letter is from, who )
            print( '  citeertitel',document.titel_citeer ) # for wetsvoorstel-dossiers, this often seems to name the law. Or a related one, see e.g. 36195
            print( '  titel',document.titel )              # for wetsvoorstel-dossiers, this seems to often name the law, plus some reason
            #print( 'versies', document.versies )
            print( '  url', document.bestand_url )
            if 0:        # It may be interesting to know the document is part of multiple dossiers and/or multiple zaken
                print( '  zaken', document.zaken )
                #nums = document.dossier_nummers
                #nums.pop(dossier.nummer)
                #if len(nums)>0:
                #  print( "  also in dossiers: %s"%nums )

            print()



== '87a7ff74-bf81-4e71-a69e-2439ce536c2c' == Dossier 36346 == 4 docs == Voorstel van wet van het lid Van Houwelingen betreffende het houden van een raadplegend referendum over het Nederlandse lidmaatschap van de Europese Unie (Wet raadplegend referendum Nederlands EU-lidmaatschap) ==
#1 GELEIDENDE_BRIEF
  onderwerp Geleidende brief
  citeertitel Wet raadplegend referendum Nederlands EU-lidmaatschap
  titel Voorstel van wet van het lid Van Houwelingen betreffende het houden van een raadplegend referendum over het Nederlandse lidmaatschap van de Europese Unie (Wet raadplegend referendum Nederlands EU-lidmaatschap)
  url https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(2416f81d-4ee0-4111-a00b-c412992ed7ac)/TK.DA.GGM.OData.Resource()

#2 VOORSTEL_VAN_WET_INITIATIEFVOORSTEL
  onderwerp Voorstel van wet
  citeertitel Wet raadplegend referendum Nederlands EU-lidmaatschap
  titel Voorstel van wet van het lid Van Houwelingen betreffende het houden van een raadplegend referendum ov